<a href="https://colab.research.google.com/github/LikeWind99/DeepLearning/blob/master/%E7%AE%80%E5%8D%95%E5%8D%B7%E7%A7%AF(%E5%89%AF%E6%9C%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import torch as t
import torchvision
from torchvision.datasets import ImageFolder
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch.nn.functional as F
import tqdm
import numpy as np
import os
from PIL import Image

In [89]:
normalize = T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
transform = T.Compose([   
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.ToTensor(),
    normalize
])
device = t.device('cuda' if t.cuda.is_available() else 'cpu')
batch_size = 8
num_works = 2
epochs = 200

In [90]:
class myDataset(Dataset):
  def __init__(self, root, train=False, transform=None):
    super(myDataset, self).__init__()
    self.train = train
    self.path = [os.path.join(root, eachDir) for eachDir in os.listdir(root)]
    self.trainset = [os.path.join(each, file) for each in self.path for file in sorted(os.listdir(each)[:int(0.8 * len(os.listdir(each)))])]
    self.testset = [os.path.join(each, file) for each in self.path for file in sorted(os.listdir(each)[int(0.8 * len(os.listdir(each))):])]
    self.transform = transform
    
  def __getitem__(self, index):
    if self.train:
      label = 0 if 'negative' in self.trainset[index] else 1
      data = Image.open(self.trainset[index]).convert("RGB")
      data = self.transform(data)
    else:
      label = 0 if 'negative' in self.testset[index] else 1
      data = Image.open(self.testset[index]).convert("RGB")
      data = self.transform(data)
    return data, label

  def __len__(self):
    if self.train:
      return len(self.trainset)
    else:
      return len(self.testset)

In [91]:
trainDataset = myDataset(r'/content/drive/MyDrive/test',train=True, transform=transform)
testDataset = myDataset(r'/content/drive/MyDrive/test',train=False, transform=transform)

trainLoader = DataLoader(dataset=trainDataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [92]:
class convNet(nn.Module):
  def __init__(self, num_class=2):
    super(convNet, self).__init__()
    self.layer1 = nn.Sequential(
      nn.Conv2d(3, 16, 3, stride=1, padding=1),
      nn.BatchNorm2d(16),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),
    )
    self.layer2 = nn.Sequential(
      nn.Conv2d(16, 32, 3, stride=1, padding=1),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),
    )
      
    self.layer3 = nn.Sequential(
      nn.Conv2d(32, 64, 3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),
    )

    self.layer4 = nn.Sequential(
      nn.Conv2d(64, 128, 3, stride=1, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU()
    )

    self.fc = nn.Sequential(
      nn.Linear(7 * 7 * 128, 1024),
      nn.ReLU(),
      nn.Dropout(0.5),
      nn.Linear(1024, num_class)
    )
    



  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc(out)
    return out

In [93]:
net = convNet()
best_loss = 1.0
total_step = len(trainLoader)
criterion = nn.CrossEntropyLoss(weight=t.from_numpy(np.array([0.1, 20.0])).float(), size_average=True).to(device)
optimizer = t.optim.Adam(params=net.parameters())
for epoch in range(epochs):
  for i, (imgs, labels) in tqdm.tqdm(enumerate(trainLoader)):
    net = net.to(device)
    optimizer.zero_grad()
    imgs = imgs.to(device)
    labels = labels.to(device)
    predict = net(imgs)
    loss = criterion(predict, labels)
    loss.backward()
    optimizer.step()
    if (i + 1) % 10 == 0:
      print(f'Epoch [{epoch}/{epochs}], Step [{i+1}/{total_step}], loss: {loss.item():.4f}')

  if loss.item() < best_loss:
    best_loss = loss.item()
    t.save(net.state_dict(), 'convNet.pth')

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
14it [00:00, 40.33it/s]

Epoch [0/200], Step [10/343], loss: 0.0000


29it [00:00, 42.22it/s]

Epoch [0/200], Step [20/343], loss: 2.5615


34it [00:00, 42.34it/s]

...

Epoch [199/200], Step [320/343], loss: 0.0022


335it [00:07, 39.03it/s]

Epoch [199/200], Step [330/343], loss: 0.2838


343it [00:07, 43.06it/s]

Epoch [199/200], Step [340/343], loss: 0.0902


In [94]:
testLoader = DataLoader(dataset=testDataset, batch_size=batch_size, drop_last=True)
net = convNet().to(device)
net.load_state_dict(t.load('convNet.pth'))
with t.no_grad():
    correct = 0
    total = 0
    cnt = 0
    for imgs, labels in testLoader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        outputs = net(imgs)
        _, predicted = t.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(f'Accuracy of the model on the test images: {100 * correct / total}%')


Accuracy of the model on the test images: 50.0%
Accuracy of the model on the test images: 62.5%
Accuracy of the model on the test images: 58.333333333333336%
Accuracy of the model on the test images: 56.25%
Accuracy of the model on the test images: 65.0%
Accuracy of the model on the test images: 70.83333333333333%
Accuracy of the model on the test images: 75.0%
Accuracy of the model on the test images: 78.125%
Accuracy of the model on the test images: 80.55555555555556%
Accuracy of the model on the test images: 82.5%
Accuracy of the model on the test images: 82.95454545454545%
Accuracy of the model on the test images: 84.375%
Accuracy of the model on the test images: 84.61538461538461%
Accuracy of the model on the test images: 85.71428571428571%
Accuracy of the model on the test images: 86.66666666666667%
Accuracy of the model on the test images: 87.5%
Accuracy of the model on the test images: 88.23529411764706%
Accuracy of the model on the test images: 88.19444444444444%
Accuracy of t

In [102]:
# trainLoader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
with t.no_grad():
  ones = t.ones(batch_size).to(device)
  correct = 0
  total = 0
  for imgs, labels in trainLoader:
      imgs = imgs.to(device)
      labels = labels.to(device)
      outputs = net(imgs)
      _, predicted = t.max(outputs.data, 1)

      total += (labels == ones).sum().item()
      correct += ((predicted == labels) & (labels == ones)).sum().item()
      print(f'Accuracy of the model on the test images: {100 * correct / total}%')

Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the model on the test images: 100.0%
Accuracy of the mode

In [73]:
|